In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import graph_tool.all as gt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from functions import *


# Graph-Tool compatibility
plt.switch_backend('cairo')
# Style
sns.set_theme(context='paper', style='white', palette='Set2')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

### Computation

In [3]:
# Load metadata
meta = get_meta()

In [4]:
filtered = []
for i, row in meta.iterrows():
    try:
        load_graph_by_id(row['SubID'])
        assert not np.isnan(row['nps_MoodDysCurValue'])  # Has NPS
        assert row['Sex'] == 'Female'
        assert row['Ethnicity'] != 'White'
    except:
        continue
    filtered.append(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}')
for i in (2, 4, 5, 6):
    print(filtered[i])

M19050 Hispanic Female, 74.0, BRAAK 5.0
M59593 Hispanic Female, 76.0, BRAAK 5.0
M83214 Hispanic Female, 83.0, BRAAK 6.0
M36634 Hispanic Female, 87.0, BRAAK 6.0


In [5]:
# Load AD Genes
with open('../../data/freeze2/attention/AD_genes.txt') as f:
    ad_genes = f.readlines()
ad_genes = [s.replace('\n', '') for s in ad_genes if len(s.replace('\n', '')) > 0]

# Graph summaries
# nps_WtLossCurValue, nps_MoodDysCurValue, Sex, nps_PsychoAgiCurValue
# TODO: Change `column` to respective heads
column_diff = 'att_D_AD_0_1'
column_data = 'att_D_no_prior_0'
coex_diff_compare_phenotype = 'nps_PsychoAgiCurValue'
diff_data_compare_phenotype = 'nps_WtLossCurValue'
individual_sample_ids = ['M19050', 'M59593', 'M83214', 'M36634']
diff_graph_summary_coex = compute_statistics(meta, 'BRAAK_AD', coex_diff_compare_phenotype, source='coexpression')
diff_graph_summary_att = compute_statistics(meta, 'BRAAK_AD', coex_diff_compare_phenotype, column=column_diff)
diff_graph_summary = compute_statistics(meta, 'BRAAK_AD', diff_data_compare_phenotype, column=column_diff)
data_graph_summary = compute_statistics(meta, 'BRAAK_AD', diff_data_compare_phenotype, column=column_data)

# Graphs
diff_g_individuals = [compute_graph(load_graph_by_id(individual_sample_ids[i], column=column_diff)) for i in range(2)]
data_g_individuals = [compute_graph(load_graph_by_id(individual_sample_ids[i], column=column_data)) for i in range(len(individual_sample_ids))]
# diff_g_individual_coex = compute_graph(load_graph_by_id(individual_sample_ids[0], source='coexpression'))
# diff_g_individual_diffusion = subset_graph(simulate_diffusion(diff_g_individual_coex, ad_genes), diff_g_individual)
# data_g_group = compute_graph(combine_graphs(get_contrast('c01x')['Control'], source='attention', column=column_data))

In [25]:
# Make main figure plots
fig = figure_diffusion(
        diff_g_individuals,
        diff_graph_summary_coex,
        diff_graph_summary_att,
        meta=meta,
        individual_sample_ids=individual_sample_ids[:2],
)
fig.savefig('../plots/diffusion.pdf', format='pdf', transparent=True, backend='cairo')
fig = figure_data_driven(
    [subset_by_hub(g, ['OPC']) for g in data_g_individuals],  # OPC myelin disruption
    diff_graph_summary,
    data_graph_summary,
    meta=meta,
    individual_sample_ids=individual_sample_ids,
)
fig.savefig('../plots/data_driven.pdf', format='pdf', transparent=True, backend='cairo')

In [38]:
# Test concatenate graph functionality
plt.clf()
concat = concatenate_graphs(*diff_g_individuals)
visualize_graph(concat)
plt.gca().axis('off')
plt.savefig('../plots/test.pdf', format='pdf', transparent=True, backend='cairo')

plt.clf()
concat = subset_by_hub(concat, ['OPC'])  # Could also subset after creation
visualize_graph(concat)
plt.gca().axis('off')
plt.savefig('../plots/test_small.pdf', format='pdf', transparent=True, backend='cairo')